In [20]:
from importlib import reload

from collections import Counter

import sage.combinat.partitions as partitions
import sage.libs.lrcalc.lrcalc as lrcalc
from sage.all import *

from sage.misc.latex import LatexExpr
from sage.combinat.composition import Compositions

all_partitions = partitions.ZS1_iterator
lrcoeff = lrcalc.lrcoef_unsafe



In [17]:
### Set Up

lambda0 = [1, 1, 1, 1]
norm_lambda0 = sum(lambda0) # norm

# print(lambda0, norm_lambda0, k_lambda0)

[1, 1, 1, 1] 4


In [18]:
### Functions for balanced diagrams

# creating a balanced corner of norm 2n
def balanced_corner( n ):
    p = []
    p.append(n)
    for i in range(0, n):
        p.append(1)
    return p
#print("Balanced corner: ", balanced_corner(1))

# counting 'relative' length from position i
def relative_length(p, i):
    rel_length = 0 
    for j in range(i+1, len(p)):
        if p[j] > 0:
            rel_length = rel_length + 1
    return rel_length 
# print("Relative length test: ", relative_length([2,1,1], 1) )

# checking if a diagram is balanced
def check_balanced( p ):
    pcopy = []
    for elt in p:
        pcopy.append(elt)
    length = len(pcopy)
    for i in range(0, length-1):
        if relative_length(pcopy, i) > pcopy[i]:
            return False
        else:
            j = i + 1
            while pcopy[i] > 0 and j < length and pcopy[j] > 0:
                pcopy[i] = pcopy[i]-1
                pcopy[j] = pcopy[j]-1
                j = j+1
    count = 0
    for i in range(0, length):
        if pcopy[i]:
            count = count + 1
    if count == 0:
        return True
    else:
        return False

# print("Balanced test: ", check_balanced([6,4,2,2,2,1,1]))

# finding balanced partitions of norm r
def balanced_partitions( norm_r ):
    all_partitions = Partitions(Integer(norm_r)).list()
    balanced_partitions_list = []
    for q in all_partitions:
        if check_balanced(q) == True:
            balanced_partitions_list.append( q )
    return balanced_partitions_list

# print("Test balanced partitions: ", balanced_partitions(10))

# checking if partiotion q fits in partition p
def check_fit(q, p):
    # check if q fits in p
    if len(q) > len(p) :
        return False
    if sum(q) > sum(p) :
        return False
    for i in range(0, len(q)):
        if p[i] < q[i] :
            return False
    return True

# finding all balanced subpartitions of norm r
def balanced_subpartitions( p, norm_r ):
    balanced = balanced_partitions( norm_r )
    subpartitions = []
    for q in balanced:
        if check_fit(q, p):
            subpartitions.append(q)
    return subpartitions


In [4]:
### Functions for injective resolutions

# finding maximal balanced subpartition
def max_subpartition( p ):
    norm = sum(p)
    maximal = []
    while len(maximal) == 0 and norm >= 0:
        maximal = balanced_subpartitions( p , norm)
        norm = norm - 1
    return maximal
# print("Maximal balanced diagram test: ", max_subpartition([3,2,2,2]))


# finding possible mu for which the Littlewood-Richardson coefficient is non-zero
def possible_mu(p, norm_p, norm_r):
    r = 2*norm_r
    norm_mu = norm_p - r
    mu_list = Partitions(Integer(norm_mu)).list()
    # print("List: ", mu_list)
    possible_mu_list = []
    balanced_gamma_list = balanced_subpartitions(p, r)
    for g in balanced_gamma_list:
        for mu in mu_list:
            # print(mu)
            N = lrcoeff( p, g, mu) 
            if N > 0:
                # print(N)
                while N > 0:
                    possible_mu_list.append( mu )
                    N = N-1
    return possible_mu_list


In [5]:
### Showing all modules in injective resolution
lambda0 = [1, 1, 1, 1]
max_list = max_subpartition(lambda0)
maxx = max_list[0]
resolution_length = sum(maxx) / 2 + 1
for r in range(0, resolution_length):
    print(r+1, " : ", possible_mu(lambda0, norm_lambda0, r))


1  :  [[1, 1, 1, 1]]
2  :  [[1, 1]]


In [6]:
d = 9
all_lambdas = Partitions(Integer(d)).list()
print("V^tensor d, d =", d, "\n")
for lambda1 in all_lambdas:
    n = sum(lambda1)
    max_list = max_subpartition(lambda1)
    maxx = max_list[0]
    print("For lambda =", lambda1, "with max balanced diagram: ", maxx)
    resolution_length = sum(maxx) / 2 + 1
    for r in range(0, int(resolution_length)):
        print(r+1, " : ", possible_mu(lambda1, n, r))


V^tensor d, d = 9 

For lambda = [9] with max balanced diagram:  []
1  :  [[9]]
For lambda = [8, 1] with max balanced diagram:  [1, 1]
1  :  [[8, 1]]
2  :  [[7]]
For lambda = [7, 2] with max balanced diagram:  [1, 1]
1  :  [[7, 2]]
2  :  [[6, 1]]
For lambda = [7, 1, 1] with max balanced diagram:  [2, 1, 1]
1  :  [[7, 1, 1]]
2  :  [[7], [6, 1]]
3  :  [[5]]
For lambda = [6, 3] with max balanced diagram:  [1, 1]
1  :  [[6, 3]]
2  :  [[5, 2]]
For lambda = [6, 2, 1] with max balanced diagram:  [2, 1, 1]
1  :  [[6, 2, 1]]
2  :  [[6, 1], [5, 2], [5, 1, 1]]
3  :  [[5], [4, 1]]
For lambda = [6, 1, 1, 1] with max balanced diagram:  [3, 1, 1, 1]
1  :  [[6, 1, 1, 1]]
2  :  [[6, 1], [5, 1, 1]]
3  :  [[5], [4, 1]]
4  :  [[3]]
For lambda = [5, 4] with max balanced diagram:  [1, 1]
1  :  [[5, 4]]
2  :  [[4, 3]]
For lambda = [5, 3, 1] with max balanced diagram:  [2, 1, 1]
1  :  [[5, 3, 1]]
2  :  [[5, 2], [4, 3], [4, 2, 1]]
3  :  [[4, 1], [3, 2]]
For lambda = [5, 2, 2] with max balanced diagram:  [2, 2,

In [7]:
from collections import Counter
d = 12
all_lambdas = Partitions(Integer(d)).list()
print("Minimal injective resolutions of $\\Gamma_{<\\lambda>}$ with $|\\lambda| = $", d, ": \\\\")
for lambda1 in all_lambdas:
    n = sum(lambda1)
    max_list = max_subpartition(lambda1)
    maxx = max_list[0]
    
    print("\\begin{minipage}{.2\\textwidth}")
    print("\\centering")
    print("\\begin{tabular}{c}")
    # print("For lambda =", lambda1, "with max balanced diagram: ", maxx)
    resolution_length = sum(maxx) / 2 + 1
    for r in range(int(resolution_length)-1, -1, -1):
        mu_list = possible_mu(lambda1, n, r)
        counter = Counter(mu_list)
        distinct_mu_list = list(counter.items())
        number_mu = len(distinct_mu_list)
        for j in range(0, number_mu):
            mu, freq = distinct_mu_list[j]
            len_mu = len(mu)
            if len_mu == 0:
                print("$\\Gamma_\\emptyset$")
            else:
                if freq >= 2:
                    print("$", freq, "\\Gamma_{(")
                else:
                    print("$\\Gamma_{(")
                for i in range(0, len_mu):
                    if i == len_mu-1:
                        print(mu[i])
                    else:
                        print(mu[i], ", ")
                if j == number_mu-1:
                    print(")}$")
                else:
                    print(")} \\oplus$")
        if r == 0:
            print("\\\\")
        else:
            print("\\\\ \\hline")
    print("\\end{tabular}\\end{minipage}\n")


Minimal injective resolutions of $\Gamma_{<\lambda>}$ with $|\lambda| = $ 12 : \\
\begin{minipage}{.2\textwidth}
\centering
\begin{tabular}{c}
$\Gamma_{(
12
)}$
\\
\end{tabular}\end{minipage}

\begin{minipage}{.2\textwidth}
\centering
\begin{tabular}{c}
$\Gamma_{(
10
)}$
\\ \hline
$\Gamma_{(
11 , 
1
)}$
\\
\end{tabular}\end{minipage}

\begin{minipage}{.2\textwidth}
\centering
\begin{tabular}{c}
$\Gamma_{(
9 , 
1
)}$
\\ \hline
$\Gamma_{(
10 , 
2
)}$
\\
\end{tabular}\end{minipage}

\begin{minipage}{.2\textwidth}
\centering
\begin{tabular}{c}
$\Gamma_{(
8
)}$
\\ \hline
$\Gamma_{(
10
)} \oplus$
$\Gamma_{(
9 , 
1
)}$
\\ \hline
$\Gamma_{(
10 , 
1 , 
1
)}$
\\
\end{tabular}\end{minipage}

\begin{minipage}{.2\textwidth}
\centering
\begin{tabular}{c}
$\Gamma_{(
8 , 
2
)}$
\\ \hline
$\Gamma_{(
9 , 
3
)}$
\\
\end{tabular}\end{minipage}

\begin{minipage}{.2\textwidth}
\centering
\begin{tabular}{c}
$\Gamma_{(
8
)} \oplus$
$\Gamma_{(
7 , 
1
)}$
\\ \hline
$\Gamma_{(
9 , 
1
)} \oplus$
$\Gamma_{(
8 , 
2

In [12]:
# --- SageMath Script ---

# Define the symmetric function space in the Schur basis
Sym = SymmetricFunctions(QQ)
s = Sym.schur()

# --- Two-partition LR coefficients ---
def lr_two(mu, nu):
    """
    Compute Littlewood-Richardson coefficients for two partitions.
    Returns a dictionary: {lambda_tuple: coefficient}.
    """
    product = s(mu) * s(nu)
    coeffs = product.coefficients(s)
    lambdas = product.support()
    
    lr_dict = {tuple(lam): int(c) for lam, c in zip(lambdas, coeffs)}
    return lr_dict

# --- Four-partition LR coefficients ---
def lr_four(mu1, mu2, mu3, mu4):
    """
    Compute Littlewood-Richardson coefficients for four partitions
    using iterated two-partition LR coefficients.
    Returns dictionary {lambda_tuple: coefficient}.
    """
    # Step 1: mu1 ⊗ mu2
    lr12 = lr_two(mu1, mu2)
    
    # Step 2: (mu1⊗mu2) ⊗ mu3
    lr123 = {}
    for nu1, c1 in lr12.items():
        lr_nu_mu3 = lr_two(nu1, mu3)
        for nu2, c2 in lr_nu_mu3.items():
            lr123[nu2] = lr123.get(nu2, 0) + c1 * c2
    
    # Step 3: (mu1⊗mu2⊗mu3) ⊗ mu4
    lr1234 = {}
    for nu2, c12 in lr123.items():
        lr_nu_mu4 = lr_two(nu2, mu4)
        for lam, c3 in lr_nu_mu4.items():
            lr1234[lam] = lr1234.get(lam, 0) + c12 * c3
    
    return lr1234

# --- Example Usage ---

# Example 1: Two partitions
mu = (2,1)
nu = (1,1)
print("Two-partition LR coefficients (c_{mu,nu}^lambda):")
lr2 = lr_two(mu, nu)
for lam, c in sorted(lr2.items()):
    print(f"λ = {lam} : c_{{{mu},{nu}}}^{lam} = {c}")

# Example 2: Four partitions
mu1 = (1,1)
mu2 = (2,1)
mu3 = (1,1)
mu4 = (1,0)
print("\nFour-partition LR coefficients (c_{mu1,mu2,mu3,mu4}^lambda):")
lr4 = lr_four(mu1, mu2, mu3, mu4)
for lam, c in sorted(lr4.items()):
    print(f"λ = {lam} : c_{{{mu1},{mu2},{mu3},{mu4}}}^{lam} = {c}")


Two-partition LR coefficients (c_{mu,nu}^lambda):
λ = (2, 1, 1, 1) : c_{(2, 1),(1, 1)}^(2, 1, 1, 1) = 1
λ = (2, 2, 1) : c_{(2, 1),(1, 1)}^(2, 2, 1) = 1
λ = (3, 1, 1) : c_{(2, 1),(1, 1)}^(3, 1, 1) = 1
λ = (3, 2) : c_{(2, 1),(1, 1)}^(3, 2) = 1

Four-partition LR coefficients (c_{mu1,mu2,mu3,mu4}^lambda):
λ = (2, 1, 1, 1, 1, 1, 1) : c_{(1, 1),(2, 1),(1, 1),(1, 0)}^(2, 1, 1, 1, 1, 1, 1) = 1
λ = (2, 2, 1, 1, 1, 1) : c_{(1, 1),(2, 1),(1, 1),(1, 0)}^(2, 2, 1, 1, 1, 1) = 3
λ = (2, 2, 2, 1, 1) : c_{(1, 1),(2, 1),(1, 1),(1, 0)}^(2, 2, 2, 1, 1) = 4
λ = (2, 2, 2, 2) : c_{(1, 1),(2, 1),(1, 1),(1, 0)}^(2, 2, 2, 2) = 2
λ = (3, 1, 1, 1, 1, 1) : c_{(1, 1),(2, 1),(1, 1),(1, 0)}^(3, 1, 1, 1, 1, 1) = 3
λ = (3, 2, 1, 1, 1) : c_{(1, 1),(2, 1),(1, 1),(1, 0)}^(3, 2, 1, 1, 1) = 8
λ = (3, 2, 2, 1) : c_{(1, 1),(2, 1),(1, 1),(1, 0)}^(3, 2, 2, 1) = 8
λ = (3, 3, 1, 1) : c_{(1, 1),(2, 1),(1, 1),(1, 0)}^(3, 3, 1, 1) = 6
λ = (3, 3, 2) : c_{(1, 1),(2, 1),(1, 1),(1, 0)}^(3, 3, 2) = 4
λ = (4, 1, 1, 1, 1) : c_{(1, 1),(2, 